In [52]:
import pandas
import re

In [53]:
SAMPLE = 'sample_0'

In [54]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
print(len(ground))
ground.head(1)

999


,lSolutions,iIndex
0,[2.14285714286],397760


In [55]:
solved = pandas.read_json(f'data/output/solved/{SAMPLE}.jsonl', lines=True)
print(len(solved))
solved.head(1)

999


,question_id,id,object,created,model,usage,choices,question,n,temperature,solved,flag
0,113024,chatcmpl-7NHX754J9stbIGK5w2pqoUuT8P0nM,chat.completion,1685783297,gpt-3.5-turbo,"{'prompt_tokens': 407, 'completion_tokens': 11...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,"{a: 20, b: 4}",OK


In [56]:
system = pandas.read_json('data/output/system_equations/sample_0.jsonl', lines=True)
system = system[['question_id', 'choices']]
system = system.rename(columns={'choices': 'system'})
system = system.set_index('question_id')
print(len(system))
system.head(1)

1000


,system
question_id,
74243,"[{'message': {'role': 'assistant', 'content': ..."


In [57]:
joined = pandas.merge(left=solved, right=ground, left_on='question_id', right_on='iIndex')
joined = joined.set_index('question_id').join(system)
joined = joined[~joined.index.duplicated(keep='first')]
print(len(joined))
joined.head(1)

998


,id,object,created,model,usage,choices,question,n,temperature,solved,flag,lSolutions,iIndex,system
question_id,,,,,,,,,,,,,,
337,chatcmpl-7NHX8nPNhsylPgMChy8Nn4EqdWrEl,chat.completion,1685783298,gpt-3.5-turbo,"{'prompt_tokens': 262, 'completion_tokens': 36...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,{a: 2.40000000000000},OK,[2.4],337,"[{'message': {'role': 'assistant', 'content': ..."


In [58]:
def extract_actual_answer(answer):
    answer = re.sub(r'\s', "", answer)
    answer = re.sub(r'\s|.*####', "", answer)
    return eval(answer)

def remove_leading_zero(number : str):
    return re.sub(r'^0+', '', number)

def extract_chatgpt_answer(answer_text):
    answer_text = answer_text.lower()
    try:
        answers = re.findall(r'the\s*answer\s*is\s*.*', answer_text)[-1]
        numbers = re.findall(r'\d+\.?\d*', answers)
        fractions = re.findall(r'\d+\/\d+', answers)
        answers = numbers + fractions

        to_return = []
        for answer in answers:
            try: to_return.append(round(eval(remove_leading_zero(answer))))
            except: pass
        return set(to_return)
    except:
        answers = re.findall(r'\d+\.?\d*', answer_text)
        if len(answers) == 0: return set()
        return set([answers[0]])

def extract_solved(solved):
    answers = re.findall(r'\d+\.?\d*', solved)
    return set([round(eval(answer), 2) for answer in answers])

def grade_response(row):
    solved = row['solved']
    solved = extract_solved(solved)
    system = row['system'][0]['message']['content']
    system = extract_chatgpt_answer(system)
    chatgpt_answer = extract_chatgpt_answer(row['question'])
    actual_answer = set(row['lSolutions'])

    A = 1 if len(chatgpt_answer.intersection(actual_answer)) > 0 else 0
    C = 1 if len(system.intersection(actual_answer)) > 0 else 0
    D = 1 if len(solved.intersection(actual_answer)) > 0 else 0

    row['A. ChatGPT gets it correct from text to final solution'] = A
    row['B. Sympy gets it correct from equations to final solution'] = 1
    row['C. ChatGPT gets it correct from equations to final solution'] = C
    row['D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT'] = D

    category_array = [A, C, D]

    offset = 8 if len(chatgpt_answer.intersection(solved)) > 0 else 0
    if category_array == [0, 0 , 0]: row['category'] = 1 + offset
    if category_array == [0, 0 , 1]: row['category'] = 2 + offset
    if category_array == [0, 1 , 0]: row['category'] = 3 + offset
    if category_array == [0, 1 , 1]: row['category'] = 4 + offset
    if category_array == [1, 0 , 0]: row['category'] = 5 + offset
    if category_array == [1, 0 , 1]: row['category'] = 6 + offset
    if category_array == [1, 1 , 0]: row['category'] = 7 + offset
    if category_array == [1, 1 , 1]: row['category'] = 8 + offset
    
    return row

In [61]:
joined = joined.apply(lambda row : grade_response(row), axis=1)
joined.to_json(f'data/output/joined/{SAMPLE}.jsonl', orient='records', lines=True)

In [ ]:
len(joined)

998

In [ ]:
def label_category(row):
    category = row.name[0]
    if category > 8: 
        row['E. Final answer and equations same'] = 1
        category -= 8
    else: row['E. Final answer and equations same'] = 0

    if category == 1: 
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Definitely cannot solve, may have got wrong rep.'''
        row['Understandable'] = '''YES'''
    elif category == 2: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Gets equations right, but solves wrong'''
        row['Understandable'] = '''YES'''
    elif category == 3: 
        row['Equations'] = '''FAIL'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Gets equations wrong, but it can solve otherwise.'''
        row['Understandable'] = '''YES'''
    elif category == 4: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Non-determinism issues as we do not know why it can't handle by itself'''
        row['Understandable'] = '''NO'''
    elif category == 5: 
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Non-determinism as we do not know how it could gotten the problem right'''
        row['Understandable'] = '''NO'''
    elif category == 6: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Can get the right equations, perhaps there is something about the equations from ground truth that mess it up'''
        row['Understandable'] = '''MAYBE'''
    elif category == 7: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''ChatGPT is lying on the equations it reports'''
        row['Understandable'] = '''MAYBE'''
    elif category == 8: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Gets everything right'''
        row['Understandable'] = '''YES'''
    return row

values = pandas.DataFrame(joined[[
    'category', 
    'A. ChatGPT gets it correct from text to final solution', 
    'B. Sympy gets it correct from equations to final solution',
    'C. ChatGPT gets it correct from equations to final solution',
    'D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT']].value_counts().sort_index())
values = values.apply(lambda row : label_category(row), axis=1)
values = values[[
    'E. Final answer and equations same',
    'count',
    'Equations',
    'Solving',
    'Understandable',
    'Theory'
]]
values

,,,,,E. Final answer and equations same,count,Equations,Solving,Understandable,Theory
category,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT,,,,,,
1,0,1,0,0,0,124,UNKNOWN,FAIL,YES,"Definitely cannot solve, may have got wrong rep."
2,0,1,0,1,0,62,SUCCEED,FAIL,YES,"Gets equations right, but solves wrong"
3,0,1,1,0,0,64,FAIL,SUCCEED,YES,"Gets equations wrong, but it can solve otherwise."
4,0,1,1,1,0,58,SUCCEED,SUCCEED,NO,Non-determinism issues as we do not know why i...
5,1,1,0,0,0,7,UNKNOWN,FAIL,NO,Non-determinism as we do not know how it could...
6,1,1,0,1,0,3,SUCCEED,FAIL,MAYBE,"Can get the right equations, perhaps there is ..."
7,1,1,1,0,0,35,SUCCEED,SUCCEED,MAYBE,ChatGPT is lying on the equations it reports
8,1,1,1,1,0,10,SUCCEED,SUCCEED,YES,Gets everything right
9,0,1,0,0,1,47,UNKNOWN,FAIL,YES,"Definitely cannot solve, may have got wrong rep."


In [ ]:
import xlsxwriter

writer = pandas.ExcelWriter('data/output/xlsx/categories.xlsx', engine='xlsxwriter')
values.to_excel(writer, sheet_name='List')

workbook = writer.book

wrap_format = workbook.add_format()
wrap_format.set_text_wrap()

list_sheet = writer.sheets['List']
list_sheet.set_column('A:H', 15, wrap_format)
pie_chart = workbook.add_chart({'type': 'pie'})

pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'ChatGPT stats'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=Chart!$A$2:$A$10",
    'values': f"=Chart!$G$2:$G$16",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}},
        {'fill': {'color': '#ffc619'}},
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('A20', pie_chart, {'x_offset': 0, 'y_offset': 0})
workbook.close()


C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$A$2:$A$10' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$G$2:$G$16' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
